<a href="https://colab.research.google.com/github/ahmedovich19/Machine-Learning-Projects/blob/master/scrapping_google_play_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -qq google-play-scraper

In [6]:
import json
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter
from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid',palette='muted', font_scale=1.2)

In [10]:
app_packages = [
    'com.anydo',
    'com.todoist',
    'com.ticktick.task',
    'com.habitrpg.android.habitica',
    'cc.forestapp',
    'com.oristats.habitbull',
    'com.levor.liferpgtasks',
    'com.habitnow',
    'com.microsoft.todos',
    'prox.lab.calclock',
    'com.gmail.jmartindev.timetune',
    'com.artfulagenda.app',
    'com.tasks.android',
    'com.appgenix.bizcal',
    'com.appxy.planner'
]

In [11]:
len(app_packages)

15

In [12]:
app_infos = []
for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)

100%|██████████| 15/15 [00:02<00:00,  5.81it/s]


In [16]:
def print_json(json_object):
  json_str = json.dumps(
      json_object,
      indent = 2,
      sort_keys=True,
      default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [18]:
print_json(app_infos[0])

{
  "adSupported": null,
  "androidVersion": "Varies",
  "androidVersionText": "Varies with device",
  "appId": "com.anydo",
  "containsAds": false,
  "contentRating": "Everyone",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "\ud83e\udd47 <b>\"#1 to do list app out there\u201d</b> - WSJ\r\n\ud83c\udfc6 <b>Editor's Choice</b> by Google\r\n\r\nOver 30M people rely on Any.do to stay organized and get more done.\r\nIt's a simple to do list app with reminders, planner & calendar - all in one.\r\n\r\n<b>\ud83e\udd47 \"A MUST HAVE APP\" (Lifehacker, NYTimes, USA TODAY).</b>\r\n\r\nAny.do is a free to-do list, planner & calendar app for managing and organizing your daily tasks, to-do lists, notes, reminders, checklists, calendar events, grocery lists and more.\r\n\r\n\ud83d\udcc5 Organize your tasks & to-do list\r\n\r\n\u2022 ADVANCED CALENDAR & DAILY PLANNER - Keep your to-do list and calendar events always at hand with our calendar widget. Any.do to-do list & pla

In [22]:
def format_title(title):
  sep_index = title.find(":") if title.find(":") != -1 else title.find('-')
  if sep_index != -1:
    title = title[:sep_index]
  return title[:10]

fig, axs = plt.subplots(2,(len(app_infos)//2), figsize=(10, 4))

for i, ax in enumerate(axs.flat):
  ai = app_infos[i]
  img = plt.imread(ai['icon'])
  ax.imshow(img)
  ax.set_title(format_title(ai['title'][:10]))
  ax.axis('off')

In [23]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.head()

,title,description,descriptionHTML,summary,summaryHTML,installs,minInstalls,score,ratings,reviews,histogram,price,free,currency,sale,saleTime,originalPrice,saleText,offersIAP,inAppProductPrice,size,androidVersion,androidVersionText,developer,developerId,developerEmail,developerWebsite,developerAddress,privacyPolicy,developerInternalID,genre,genreId,icon,headerImage,screenshots,video,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,recentChanges,recentChangesHTML,editorsChoice,appId,url
0,"Any.do: To do list, Task, Reminders & Planner","🥇 <b>""#1 to do list app out there”</b> - WSJ\r...",🥇 <b>&quot;#1 to do list app out there”</b> - ...,To-do list & Tasks ✅ Calendar 📅ֿ Reminder 🔔 ...,To-do list &amp; Tasks ✅ Calendar 📅ֿ Reminde...,"10,000,000+",10000000,4.350008,364313,129003,"[33132, 11110, 18234, 34471, 267366]",0,True,USD,False,None,None,None,True,$0.99 - $83.88 per item,Varies with device,Varies,Varies with device,Any.do To-do list & Calendar,5304780265295461149,feedback+androidtodo@any.do,https://www.any.do,"Any.do Inc.\n\n6 Agripas Street, Tel Aviv\n62...",https://www.any.do/privacy,5304780265295461149,Productivity,PRODUCTIVITY,https://play-lh.googleusercontent.com/zgOLUXCH...,https://play-lh.googleusercontent.com/umhDP6ph...,[https://play-lh.googleusercontent.com/cGoZDZR...,None,None,Everyone,None,None,False,"Nov 10, 2011",1614496999,Varies with device,"- Dark mode is finally here, hooray! \r\n- Rec...","- Dark mode is finally here, hooray! <br>- Rec...",False,com.anydo,https://play.google.com/store/apps/details?id=...
1,"Todoist: To-Do List, Tasks & Reminders","🏆 <b>2020 Editor's Choice</b> by Google\r\n🥇 ""...",🏆 <b>2020 Editor&#39;s Choice</b> by Google<br...,"""The best to-do list"" — The Verge. Tasks ✅ Rem...",&quot;The best to-do list&quot; — The Verge. T...,"10,000,000+",10000000,4.555631,219895,66813,"[8869, 3617, 7733, 35917, 163759]",0,True,USD,False,None,None,None,True,$0.99 - $36.00 per item,Varies with device,Varies,Varies with device,Doist,6748661820728732000,support@todoist.com,http://todoist.com,"Doist\n201 Mission Street, Suite 1200\nSan Fra...",https://todoist.com/Help/viewPrivacyPolicy,6748661820728732000,Productivity,PRODUCTIVITY,https://play-lh.googleusercontent.com/GK0SwEBV...,https://play-lh.googleusercontent.com/sAjzU1fU...,[https://play-lh.googleusercontent.com/cealfl9...,None,None,Everyone,None,None,False,"Nov 18, 2012",1614609241,Varies with device,⭐️ Personalize your mobile icon\r\nMake Todois...,⭐️ Personalize your mobile icon<br>Make Todois...,True,com.todoist,https://play.google.com/store/apps/details?id=...
2,"TickTick: ToDo List Planner, Reminder & Calendar",🏆<b>Editor's Choice</b> by Google\r\n🥇<b>Great...,🏆<b>Editor&#39;s Choice</b> by Google<br>🥇<b>G...,Task & time manager by Editor’s Choice🏆. Get t...,Task &amp; time manager by Editor’s Choice🏆. G...,"1,000,000+",1000000,4.608742,74721,27310,"[2837, 1145, 2409, 9629, 58701]",0,True,USD,False,None,None,None,True,$0.99 - $27.99 per item,Varies with device,Varies,Varies with device,Appest Inc.,7269704759597705101,ticktick.com@gmail.com,https://ticktick.com,2035 Sunset Lake Road\nNewark\nDelaware\nUS,https://ticktick.com/about/privacy,7269704759597705101,Productivity,PRODUCTIVITY,https://play-lh.googleusercontent.com/YSlqf3WB...,https://play-lh.googleusercontent.com/m-kwaaJE...,[https://play-lh.googleusercontent.com/baCPuc4...,None,None,Everyone,None,None,False,"Jun 19, 2013",1614589211,Varies with device,✍️ Smoother experience of Markdown.\r\n📱 Adapt...,✍️ Smoother experience of Markdown.<br>📱 Adapt...,True,com.ticktick.task,https://play.google.com/store/apps/details?id=...
3,Habitica: Gamify Your Tasks,Treat your life like a game to stay motivated ...,Treat your life like a game to stay motivated ...,Treat your life like a game to stay motivated ...,Treat your life like a game to stay motivated ...,"1,000,000+",1000000,4.270766,18534,7718,"[1439, 709, 1349, 2929, 12108]",0,True,USD,False

In [24]:
app_infos_df.to_csv('apps.csv',index=None,header=True)

In [29]:
app_reviews = []
for ap in tqdm(app_packages):
  for score in range(1, 6):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs = reviews(
          ap,
          lang='en',
          country = 'us',
          sort=sort_order,
          count=200 if score == 3 else 100,
          filter_score_with=score
      )[0]
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 15/15 [01:14<00:00,  4.94s/it]


In [30]:
print_json(app_reviews[0])

{
  "appId": "com.anydo",
  "at": "2021-03-05 00:37:04",
  "content": "Was simply looking for a widget that could sync all my calendars from all my email addresses. Sadly, this can't, despite what it claims in the description. It seems like you need to sign up for another service (Zapier or something).",
  "repliedAt": "2021-03-07 10:03:08",
  "replyContent": "Any.do\u2019s Calendar Integration syncs events to and from your native calendars, there is no need to sign in to another service. You will first need to add your calendar on your mobile device. Please see instructions: support.any.do/calendar-setup. If you need further assistance, kindly contact us at- https://www.any.do/contact_form.\n",
  "reviewCreatedVersion": "5.10.0.8",
  "reviewId": "gp:AOqpTOEdg6yUOvcZHEm6vmnCD7vXQW7wuauIJaKhKtwZEd4mf34DMuydW0n0Jjg5ccfqzzRvQ_Cm-4siExWlfA",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 0,
  "userImage": "https://play-lh.googleusercontent.com/-jhjI7vIOy0Q/AAAAAAAAAAI/AAA

In [31]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.shape

(16409, 12)

In [32]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOEdg6yUOvcZHEm6vmnCD7vXQW7wuauIJaKhKtw...,Uchihasocksy,https://play-lh.googleusercontent.com/-jhjI7vI...,Was simply looking for a widget that could syn...,1,0,5.10.0.8,2021-03-05 00:37:04,Any.do’s Calendar Integration syncs events to ...,2021-03-07 10:03:08,most_relevant,com.anydo
1,gp:AOqpTOHTeNGxl9Zhlz-BM6ybraEsnVg0BLDubF8E_fQ...,kelvin ho,https://play-lh.googleusercontent.com/-rSnBD4Y...,Very very bad experience with the management! ...,1,5,5.10.0.8,2021-02-26 04:17:49,"Hi Kelvin, we've checked our records and could...",2021-02-28 09:36:51,most_relevant,com.anydo
2,gp:AOqpTOH5OkwJH-nRT4nopD_JNTIYpWs1xWzOsFE-pn7...,Charles Green,https://play-lh.googleusercontent.com/a-/AOh14...,"Ugh! After years of using this app, I am so ve...",1,10,5.9.0.2,2021-02-17 19:10:03,"From checking our records, the issue you've re...",2021-02-18 11:22:00,most_relevant,com.anydo
3,gp:AOqpTOG4LKg25ejg1Hw1TWs6_gIXhpwJRZZpXssMygx...,Andi Tan,https://play-lh.googleusercontent.com/-vozop-l...,I'm a new user. I think this app's great. I tr...,1,13,5.9.0.2,2021-02-07 11:54:54,The Quick-add bar on the device's Notification...,2021-02-09 09:19:18,most_relevant,com.anydo
4,gp:AOqpTOGQ7NShvSVr9Vqz67MSeTN_3rkBuYtQRtsyAo5...,Drew Dayman,https://play-lh.googleusercontent.com/a-/AOh14...,Bought it to use with Alexa. After multiple at...,1,1,5.9.0.2,2021-02-12 20:35:18,None,NaT,most_relevant,com.anydo


In [34]:
app_reviews_df.to_csv('reviews.csv',index=None,header=True)